In [1]:
import sys
import itertools

import numpy as np
import pandas as pd
import scipy.signal
from scipy.signal import savgol_filter
from scipy.stats import zscore
from tqdm import tqdm

import neuroseries as nts


import time
import matplotlib.pyplot as plt

import seaborn as sns 
import bk.load
import bk.compute
import bk.plot

import os

plt.rcParams['svg.fonttype'] = 'none'
%matplotlib qt


In [2]:
bk.load.current_session_linux()
states = bk.load.states()
transitions = bk.compute.transition(states,['sws','Rem'],50)

Rat : 8 on day : 6
Working with session Rat08-20130713 @ /mnt/electrophy/Gabrielle/GG-Dataset-Light/Rat08/Rat08-20130713


In [3]:
chans = {'left_bla': bk.load.bla_channels()['left'],
        'right_bla': bk.load.bla_channels()['right'],
        'hpc': bk.load.ripple_channel()}

lfp = bk.load.lfp(chans['right_bla'])


In [11]:
f, t, Sxx = scipy.signal.spectrogram(
    lfp, 1250, nperseg=10000, noverlap=8000)

band = {'delta': [0.1, 4],
        'theta': [5, 10],
        'gamma': [60, 80]}

filter = {}
power = {}
for b, (i, j) in band.items():
    print(b)
    filt = (i < f) & (f < j)
    filter.update({b: filt})

    power.update({b: nts.Tsd(t, np.mean((Sxx[filt, :]), 0), time_units='s')})

for b in power:
    power[b] = power[b].restrict((states['Rem']))


delta
theta
gamma


In [12]:
fig,ax = plt.subplots(5,1,sharex=True)
plt.sca(ax[0])
bk.plot.intervals(transitions[0],'red')
plt.plot(lfp.as_units('s'))
plt.sca(ax[1])
bk.plot.intervals(states['sws'],'grey')
bk.plot.intervals(states['Rem'])

ax[2].plot(zscore(power['delta'].as_units('s')))
ax[3].plot(zscore(power['theta'].as_units('s')))
plt.sca(ax[3])
bk.plot.intervals(theta_bouts)
ax[4].plot(zscore(power['gamma'].as_units('s')))

# for i in range(2,5):ax[i].set_ylim(-3,3)


In [9]:
theta_bouts = bk.load.intervals('BLA_REM_theta_bouts')

In [8]:
bk.load.computed_intervals()

['sleep_scoring.csv',
 'sleep_wake.csv',
 'runs.csv',
 'BLA_REM_theta_bouts.csv',
 'sleep_scoring_automatic.csv',
 'laps.csv']